In [ ]:
import numpy as np
from my_functions import *
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
split_converter = lambda s: s.split()
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))
tile_name = lambda s: s[:8]

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_2.csv', sep=',', header=1,
    converters={0: tile_name, 1: split_converter, 2: split_converter, 3: sum_flags,
    4: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0]

In [ ]:
params_tiles = []
for tile in ['AEGIS001', 'AEGIS002', 'AEGIS003', 'AEGIS004']:
    err_fit_params = []

    pm_flx = np.stack(
        cat['FLUX_AUTO'][np.where(cat['name'] == tile)[0]].to_numpy()
    ).astype(float).T * 1e-19
    pm_err = np.stack(
        cat['FLUX_RELERR_AUTO'][np.where(cat['name'] == tile)[0]].to_numpy()
    ).astype(float).T

    for nb_c in range(60):
        mask = (pm_flx[nb_c] > 0) & (pm_err[nb_c] > 0)
        err_fit_params.append(
            np.polyfit(np.log10(pm_flx[nb_c, mask]), np.log10(pm_err[nb_c, mask]), 2)
        )
    params_tiles.append(err_fit_params)

In [ ]:
tiles = ['AEGIS001', 'AEGIS002', 'AEGIS003', 'AEGIS004']
for i in range(4):
    tile = tiles[i]
    pm_flx = np.stack(
        cat['FLUX_AUTO'][np.where(cat['name'] == tile)[0]].to_numpy()
    ).astype(float).T * 1e-19
    pm_err = np.stack(
        cat['FLUX_RELERR_AUTO'][np.where(cat['name'] == tile)[0]].to_numpy()
    ).astype(float).T

    fig, ax = plt.subplots(figsize=(11, 9))

    nb_c = 12
    a = params_tiles[i][nb_c][0]
    b = params_tiles[i][nb_c][1]
    c = params_tiles[i][nb_c][2]

    ax.scatter(np.log10(pm_flx[nb_c]), np.log10(pm_err[nb_c]))
    x_Arr = np.linspace(-23, -14, 100)
    ax.plot(x_Arr, a * x_Arr**2 + b * x_Arr + c, c='C1')

    plt.show()

In [ ]:
np.save('npy/err_fit_params_minijpas', np.array(err_fit_params))